In [1]:
import json
import pandas as pd
from time import time
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql, load_my_columns, \
                        get_numbers,map_dff_to_my_columns, max_number_find, \
                        select

In [2]:
start = time()
# get_nums_table='bronze.katm_077_contingent_liabilities'
# foreign_key = 'contingent_liabilities_qty'
write_to_table = 'bronze.katm_077_contingent_liabilities_offline'
columns_file = 'katm_077_contingent_liabilities_columns.txt'
columns = load_my_columns(columns_file)

In [3]:
client = get_mongo_client()
db = client['katm']
task_collection = db['katm_cache']

query = {
    'reports.077.contingent_liabilities.contingent_liability': {'$exists': True}
    # ,'number': {'$eq': 5651095}
    # ,'number': {'$gt': max_num}
    # ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}   
}
projection = {
    '_id': 1,
    'reports.077.contingent_liabilities.contingent_liability': 1
}

In [4]:
docs = task_collection.find_one(query, projection)

In [5]:
docs

{'_id': ObjectId('65616d529967f3feb5c98968'),
 'reports': {'077': {'contingent_liabilities': {'contingent_liability': {'max_uninter_overdue_percent': '',
     'overdue_debt_sum': '',
     'contract_closing_date': '',
     'contract_id': '',
     'contract_status': '',
     'max_current_uninter_overdue_percent': '',
     'branch': '',
     'amount_change': '',
     'percent': '',
     'contract_end_date': '',
     'contract_date': '',
     'total_debt_sum': '',
     'currency': '',
     'max_current_overdue_change': '',
     'org_name': '',
     'percent_change': '',
     'max_overdue_principal_change': '',
     'amount': '',
     'subject_type': '',
     'org': '',
     'overdue_debt_sum_change': '',
     'max_overdue_principal': '',
     'max_current_overdue': '',
     'max_current_uninter_overdue_percent_change': '',
     'max_uninter_overdue_percent_change': '',
     'total_debt_sum_change': '',
     'loan_sibject': ''}}}}}

In [6]:
max_date = select('select max(cast(contract_date as date)) from bronze.katm_077_contracts_offline')[0].strftime('%Y-%m-%d')

In [7]:
max_date

'2024-02-26'

In [8]:
client = get_mongo_client()
db = client['katm']
task_collection = db['katm_cache']

query = {
    'reports.077.contracts.contract': {'$exists': True}
    # ,'number': {'$eq': 8141290}
    # ,'number': {'$gt': max_num}
    # ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}
    , 'reports.077.contracts.contract.contract_date':{'$gte':max_date}
}
projection = {
    '_id': 1,
    'reports.077.contracts.contract': 1
}
docs = task_collection.find_one(query, projection)

In [9]:
docs

{'_id': ObjectId('65dc153cd3270e2feaf70117'),
 'reports': {'077': {'contracts': {'contract': [{'reserve_bal_change': '',
      'contract_id': '617909',
      'security_qty': '1',
      'contract_status': '1',
      'revised_principal_sum': '',
      'offbalance_princial_sum_change': '',
      'branch': '01183',
      'securities': {'security': {'pinfl': '-',
        'amount': '832000000',
        'subject_type': '',
        'contract_id': '617909',
        'inn': '',
        'security_change': '2024-02-23',
        'contract_date': '',
        'juridical_status': '2',
        'name': "ABILXASI'MOV SULTANBEK YUSUP-ULI'",
        'currency': '000',
        'consent_date': '22.02.24',
        'security_type': '73',
        'consent_id': '617909'}},
      'class_asset_quality': '1',
      'forecasted_schedule': {'forecasted_schedule_change': '2024-02-26',
       'forecasted_schedule_change_qty': '1',
       'forecasted_payment': [{'forecasted_payment_period': '2024-02',
         'principal

In [12]:
import json
import pandas as pd
from time import time
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql, load_my_columns, \
                        map_dff_to_my_columns

In [13]:
start = time()
write_to_table = 'bronze.asbt_list_online_microloan'
columns_file = 'asbt_credit_list_online_microloan.txt'
columns = load_my_columns(columns_file)

In [14]:
client = get_mongo_client()
db = client['task']
task_collection = db['task']

query = {
    'data.asbt_credit_list.return.object': {'$exists': True}
}
projection = {
    'number': 1,
    'data.asbt_credit_list.return.object': 1
}

In [15]:
docs = task_collection.find_one(query, projection)

In [16]:
docs

{'_id': ObjectId('63ef578b59db4840e879875a'),
 'number': 79,
 'data': {'asbt_credit_list': {'return': {'object': [{'idMfo': '937',
      'id': '93700014295813',
      'globId': '93700002157049',
      'nikiId': '0002157049',
      'numDog': '13+1',
      'dateDog': '2023-02-14 00:00:00.0',
      'client': 'MUXITDINOVA NARGIZA ABDULAZIZOVNA',
      'namAcc': 'Ипотека ва истеъмол кредитлари учун (ФИО)',
      'idClient': '4893193',
      'accNew': '14901000662038997801',
      'status': '0',
      'inspect': '1180',
      'kodVal': '0',
      'kodSost': '3',
      'procSs': '28',
      'procPr': '42',
      'basPer': '365',
      'dateVydD': '2023-02-14 00:00:00.0',
      'datePogD': '2025-02-13 00:00:00.0',
      'sumRas': '100000000',
      'mejBank': '0',
      'summa': '100000000',
      'sumObesp': '130000000',
      'srok': '3',
      'vidCred': '32',
      'celSsudy': '60024',
      'typeOpesp': '51',
      'istCred': '11',
      'numTran': '801',
      'charVal': 'UZS',
      'na

In [17]:
rows=[]

number = docs.get('number')
fields = docs.get('data', {}).get('asbt_credit_list', {}).get('return', {}).get('object', [])
if isinstance(fields, dict):
    fields = [fields]
elif fields is None:
    fields = []
for field in fields:
    # filtered_field = {k: v for k, v in field.items() if not isinstance(v, (dict, list))}
    row = {'number': number, **field}
    rows.append(row)

In [18]:
df = pd.DataFrame(rows)

In [19]:
df

,number,idMfo,id,globId,nikiId,numDog,dateDog,client,namAcc,idClient,...,repaymentAccountRest,summaGashP,summaPr,summaPrOsn,summaPrProc,summaGNext,dateGash,summaGMin,saldoIn,saldoF
0,79,937,93700014295813,93700002157049,0002157049,13+1,2023-02-14 00:00:00.0,MUXITDINOVA NARGIZA ABDULAZIZOVNA,Ипотека ва истеъмол кредитлари учун (ФИО),4893193,...,0,230136,0,0,0,5488848,2023-03-03 00:00:00.0,5488848,100000000,100230136
1,79,937,93700014300760,93700002157486,0002157486,13+1,2023-02-15 00:00:00.0,MUXITDINOVA NARGIZA ABDULAZIZOVNA,Ипотека ва истеъмол кредитлари учун (ФИО),4893193,...,0,153424,0,0,0,5488848,2023-03-03 00:00:00.0,5488848,100000000,100153424


In [20]:
df.columns

Index(['number', 'idMfo', 'id', 'globId', 'nikiId', 'numDog', 'dateDog',
       'client', 'namAcc', 'idClient', 'accNew', 'status', 'inspect', 'kodVal',
       'kodSost', 'procSs', 'procPr', 'basPer', 'dateVydD', 'datePogD',
       'sumRas', 'mejBank', 'summa', 'sumObesp', 'srok', 'vidCred', 'celSsudy',
       'typeOpesp', 'istCred', 'numTran', 'charVal', 'namStatus', 'nameIns',
       'nameType', 'saldo', 'repaymentAccount', 'repaymentAccountRest',
       'summaGashP', 'summaPr', 'summaPrOsn', 'summaPrProc', 'summaGNext',
       'dateGash', 'summaGMin', 'saldoIn', 'saldoF'],
      dtype='object')

In [1]:
from katm_utils import max_number_find_with_cast

In [3]:
max_num = max_number_find_with_cast('bronze.asbt_list_online_microloan')

In [4]:
max_num

970220

In [6]:
from katm_connections import get_mongo_client

In [7]:
client = get_mongo_client()
db = client['task']
task_collection = db['task']

query = {
    'data.asbt_credit_list.return.object': {'$exists': True}
    ,'number': {'$gt': max_num}
}
projection = {
    'number': 1,
    'data.asbt_credit_list.return.object': 1
}

In [8]:
doc = task_collection.find_one(query,projection)

In [9]:
doc

{'_id': ObjectId('655d82b68c5d13926a656719'),
 'number': 970221,
 'data': {'asbt_credit_list': {'return': {'object': [{'idMfo': '122',
      'id': '12200016519976',
      'globId': '12200002236328',
      'nikiId': '0002236328',
      'numDog': '140678/2023',
      'dateDog': '2023-05-25 00:00:00.0',
      'client': 'XOLBOYEV BUNYOD NORBOY O‘G‘LI',
      'namAcc': 'Ипотека ва истеъмол кредитлари учун (XOLBOYEV BUNYOD NORBOY O‘G‘LI)',
      'idClient': '7802961',
      'accNew': '14901000463508792001',
      'status': '0',
      'inspect': '8860',
      'kodVal': '0',
      'kodSost': '3',
      'procSs': '24',
      'procPr': '28.8',
      'basPer': '365',
      'dateVydD': '2023-05-25 00:00:00.0',
      'datePogD': '2026-05-22 00:00:00.0',
      'sumRas': '4670000000',
      'mejBank': '0',
      'summa': '4670000000',
      'sumObesp': '5837500000',
      'srok': '3',
      'vidCred': '32',
      'celSsudy': '60024',
      'typeOpesp': '41',
      'istCred': '11',
      'numTran': '8

In [10]:
from datetime import datetime

In [11]:
print(datetime(2023,11,1))

2023-11-01 00:00:00


In [12]:
import json
import pandas as pd
from time import time
from datetime import datetime
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql_tqdm, load_my_columns, \
                        map_dff_to_my_columns, max_number_find_with_cast


In [13]:
client = get_mongo_client()
db = client['task']
task_collection = db['task']

date_filter = datetime(2023,11,1)

In [16]:
date_filter

datetime.datetime(2023, 11, 1, 0, 0)

In [75]:
query = {
    'data.katm_077.result': {'$exists': True},
    # 'number': {'$eq': 9447946},
    # 'created': {'$gt': date_filter},
    'data.katm_077.resutl.is_ok': 'false'
}

projection = {
    'number': 1,
    'created': 1,
    'request.clientId': 1,
    'data.katm_077.result': 1
}


In [76]:
doc = task_collection.find_one(query, projection)

In [74]:
doc

{'_id': ObjectId('67c697b9d63e2511c45a1c8c'),
 'request': {'clientId': '4271524'},
 'created': datetime.datetime(2025, 3, 4, 6, 3, 37, 74000),
 'number': 9447946,
 'data': {'katm_077': {'result': {'is_ok': True,
    'data': {'expired_summa': 0,
     'is_exists_stop_factor': False,
     'summa_other_mikrozaym': 5840742.92,
     'summa_other_avtocredit': 83327189.73,
     'summa_other_ipoteka': 0,
     'today_payment': 4776690,
     'loan_summa': 91254493,
     'num_all_credits_ib': 3,
     'num_all_other': 5,
     'num_all_IB': 3,
     'bureau_segment': 'ib',
     'num_active_micro_ib': 1,
     'sum_active_micro_ib': 5840742.92,
     'scoring_level': 'ХОРОШИЙ',
     'scoring_class': 'B1',
     'scoring_grade': 322,
     'score_cutoff': 180,
     'current_dlq_bureau': 0,
     'current_dlq_total': 0}}}}}

In [56]:
number = doc.get('number')

In [57]:
number

9447946

In [58]:
clientId = doc.get('request').get('clientId')

In [59]:
clientId

'4271524'

In [63]:
is_ok = doc.get('data', {}).get('katm_077', {}).get('result', {}).get('is_ok')

In [64]:
is_ok

True

In [65]:
rows=[]
fields = doc.get('data', {}).get('katm_077', {}).get('result', {}).get('data',{})
if isinstance(fields, dict):
    fields = [fields]
elif fields is None:
    fields = []
for field in fields:
    # filtered_field = {k: v for k, v in field.items() if not isinstance(v, (dict, list))}
    row = {'number': number, 'clientId': clientId, 'is_ok':is_ok, **field}
    rows.append(row)

In [66]:
df = pd.DataFrame(rows)

In [67]:
df

,number,clientId,is_ok,expired_summa,is_exists_stop_factor,summa_other_mikrozaym,summa_other_avtocredit,summa_other_ipoteka,today_payment,loan_summa,...,num_all_IB,bureau_segment,num_active_micro_ib,sum_active_micro_ib,scoring_level,scoring_class,scoring_grade,score_cutoff,current_dlq_bureau,current_dlq_total
0,9447946,4271524,True,0,False,5840742.92,83327189.73,0,4776690,91254493,...,3,ib,1,5840742.92,ХОРОШИЙ,B1,322,180,0,0


In [72]:
query = {
    'data.katm_077.result': {'$exists': True}
    # ,'number': {'$gt': max_num}
    ,"created": {"$gt": date_filter}
    # ,"created":{"$lt": datetime(2023,12,15)}
    ,"katm_077.result.is_ok": {"$in": [False, "false"]}
}
projection = {
    'number': 1,
    'created': 1,
    'request.clientId': 1,
    'katm_077.result.is_ok':1,
    'data.katm_077.result.error': 1
}

In [73]:
doc = task_collection.find_one(query, projection)

KeyboardInterrupt: 

In [77]:
import json
import pandas as pd
from time import time, sleep
from datetime import datetime
from pymongo.errors import CursorNotFound
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql_tqdm

In [78]:
write_to_table = 'bronze.hints'
date_filter = datetime(2023, 11, 1)
query = {
    'data.katm_077.result': {'$exists': True}
    ,"created": {"$gt": date_filter}
}
projection = {
    'number': 1,
    'data.katm_077.result': 1
}